## Step 1 - Memanggil Beberapa Library Dan Module Yang Dibutuhkan


In [15]:
import numpy as np
import keras
from keras.models import model_from_json, Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD , Adam
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

**Untuk Pengerjaan Pada Sistem Operasi Berbasis Linux Hal In Perlu Dilakukan**

In [2]:
_ROOTPATH = "/home/galgadot/Documents/Skripsi/FlappyBot/"
_LEARNING_RATE = 1e-4

##### Step Info!

  - Menyiapkan sebuah fungsi yang mengembalikan sebuah model berdasarkan arsitektur yang akan digunakan.
  - Model kembalian sudah memiliki struktur dan tinggal digunakan untuk compile serta fit (dalam tahap pelatihan)

In [3]:
""" Build Model """

def createModel():
    print("Now we build the model")
    model = Sequential()
    model.add(Convolution2D(32, 8, 8, subsample=(4, 4), border_mode='same',input_shape=(80,80,4)))  #80*80*4
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 4, 4, subsample=(2, 2), border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode='same'))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(2))
    adam = Adam(lr=_LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam, metrics=['accuracy'])
    print("We finish building the model")
    return model


##### Step Info!

  - Menyiapkan bebrapa variable aygn digunakan, batch_size, jumlah epoch untuk pelatihan, serta jumlah output tindakan (class), menyiapkan varian jumlah data untuk pelatihan pada DataSize
  - Memanggil data X dan data Y ( X adalah fitur, dan Y adalah target (class))

In [16]:
""" Initializing Data """

batch_size = 32
epochs = 20
num_classes = 2
DataSize = "1000"
data_x = np.load(_ROOTPATH+'initial_data/Data '+DataSize+'/DataX.npy')
data_y = np.load(_ROOTPATH+'initial_data/Data '+DataSize+'/DataY.npy')

train_x = data_x
print(train_x.shape)
train_y = data_y
train_y = to_categorical(train_y)
print(train_y.shape)
# train_x = train_x.astype('float32')
# train_x = train_x / 255.



(1000, 80, 80, 4)
(1000, 2)


**Membagi data untuk training dan validasi, vaidasi dibutuhkan untuk melihat hasil evaluasi model ditiap epoch saat pelatihan**

In [17]:
""" Data For Train """
train_x,valid_x,train_label,valid_label = train_test_split(train_x, train_y, test_size=0.2, random_state=13)
train_x.shape,valid_x.shape,train_label.shape,valid_label.shape

((800, 80, 80, 4), (200, 80, 80, 4), (800, 2), (200, 2))

##### Step Info!
  - Memanggil fungsi pembuat model cnn yang telah dibuat diatas tadi
  - Melihat detail cnn arsitektur yang dibut dengna memanggil fungsi **summary**
  - lalu melakuakn training sesuai ketentuan keras, dengna menggunakn _**model.fit(x,y,batch,epoch,options)**_
  

In [18]:
""" Create Model And View Summary """
flappy_model = createModel()
flappy_model.summary()

Now we build the model
Instructions for updating:
Colocations handled automatically by placer.


/usr/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (8, 8), input_shape=(80, 80, 4..., strides=(4, 4), padding="same")`
  
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (4, 4), strides=(2, 2), padding="same")`
  
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), strides=(1, 1), padding="same")`
  # Remove the CWD from sys.path while we load stuff.


We finish building the model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
activation_1 (Activation)    (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        32832     
_________________________________________________________________
activation_2 (Activation)    (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 64)        36928     
_________________________________________________________________
activation_3 (Activation)    (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)      

In [ ]:
""" Starting Training Model """
flappy_train = flappy_model.fit(train_x, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_x, valid_label))


##### Step Info!
 - Setelah melakuakn proses pelatihan diatas
 - Flappy model akan menyimpan struktur arsitekturnya dalam bentuk json. hal ini akan membantu dalam proses pengujian tidak perlu lagi membuat model dalam arsitektur yang sama.
 - untuk bobot dari model sendiri akan disimpan dalam bentuk .h5 atau h5py
  

In [ ]:
""" Save Model By Name Using JSON """
""" Serialize model to JSON """
model_json = flappy_model.to_json()
with open("saved_networks/saved_"+DataSize+"model.json", "w") as json_file:
    json_file.write(model_json)
""" serialize weights to HDF5 """
flappy_model.save_weights("saved_networks/saved_"+DataSize+"model.h5")
print("Saved model to disk")